# This script will evaluate the dataset created and identify the speaker
Note:
- If you doesn't have the dataset created please run word_identification.ipynb

## Install required packages
Run the next cell only if you don't have the packages already installed

In [ ]:
! pip install scipy matplotlib sounddevice wave playsound tqdm librosa pyphen opencv-python praat-parselmouth pyttsx3 pydub ffmpeg

## <center> <span style='color :blue' >Sound Analysis</span>

## Import all required libraries

In [1]:
# Run this cell to import all the required libraries in code below
from librosa import load, get_duration, display, feature, magphase, frames_to_time, onset, autocorrelate, effects, stft, amplitude_to_db, times_like, pyin
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score, classification_report
from scipy.signal import lfilter, lfilter_zi, filtfilt, butter, freqz, spectrogram, get_window, find_peaks
from sklearn.model_selection import train_test_split
import pyphen, math, time, glob, os, cv2, pickle, parselmouth
from librosa.util import fix_length
from parselmouth.praat import call
# from pydub import AudioSegments
from os.path import join, exists
import matplotlib.pyplot as plt
import IPython.display as ipd
from sklearn.svm import SVC
import scipy.fftpack as fft
from math import log as ln
from tqdm import tqdm
from math import exp
import pandas as pd
import numpy as np
import pyttsx3 

# Convert audio extension from m4a to wav

In [ ]:
m4a_file = '20211210_151013.m4a'
wav_filename = r"F:\20211210_151013.wav"

track = AudioSegment.from_file(m4a_file,  format= 'm4a')
file_handle = track.export(wav_filename, format='wav')

### Read and View the graph of  audio

In [ ]:
def read_audio_signal(audio_file):    
    audio, fs = load(audio_file) # read audio file with fs = 22050 in mono mode

    padded_audio = fix_length(audio, size=4*fs) # set the audio with 4 seconds
    
    duration_original = get_duration(y=audio, sr=fs) # get the duration of the audio in seconds
    duration_padded = get_duration(y=padded_audio, sr=fs) # get the duration of the audio padded in seconds
    
    duration_time_original = np.linspace(0, duration_original, audio.shape[0]) # create time vector to x-scale in graph
    duration_time_padded = np.linspace(0, duration_padded, padded_audio.shape[0]) # create time vector to x-scale in graph

    n_points_original = int(fs * duration_original) # signal size in points original
    n_points_padded = int(fs * duration_padded) # signal size in points padded
    
    return fs, audio, padded_audio, duration_time_original, duration_time_padded, n_points_original, n_points_padded
        
def show_orginal_audio(filename): # show original wave of each word
    fs, audio_orig, audio_pad,time_orig, time_paded, _, _ = read_audio_signal(filename) # read audio example
    
    # define ghapics parameters 
    plt.rcParams['figure.figsize'] = [15, 7]
    plt.tight_layout(pad=3.0) 
    fig, ax = plt.subplots(ncols=2, sharex=False)

    fig.suptitle(filename.split('/')[1] + ' saying ' +  filename.split('/')[-2], fontsize = 16, fontweight = 'bold')
    
    ax[0].plot(time_orig ,audio_orig) # show graph with original audio signal
    ax[0].set(title='Original audio')
 
    ax[1].set(title ='Padded audio')
    ax[1].plot(time_paded ,audio_pad) # show graph with padded audio signal
    
    for ax in ax.flat:
        ax.set(xlabel='Time [s]', ylabel='Amplitude')
    plt.show()   
    

show_orginal_audio('Dataset/Andre/casa/0.wav') 

## Filter the original signal between typical voice frequencies


In [ ]:
def filter_original_voice(filename):

    filtered_audio = [] # initialize list
    
    fs, _, audio_readed, _, time_paded, _, _ = read_audio_signal(filename) # read audio example
          
    nyq = 0.5 * fs # nyquist frequency

    # normalize cutoff frequency
    low = 60 / nyq 
    high = 7000 / nyq 

    b, a = butter(6, [low, high], btype='band') # define 5th order band-pass butterworth filter 

    w, h = freqz(b, a) # compute frequency response
    
    # show frequency response
    plt.plot((fs * 0.5 / np.pi) * w, 20 * np.log10(abs(h)), 'b')
    plt.ylabel('Amplitude [dB]', color='b')
    plt.xlabel('Frequency [rad/sample]')   
    plt.title('Frequency Response',  fontsize = 16, fontweight = 'bold')  
    plt.grid(True)
    plt.xticks(range(0, 12000, 1000))
    plt.ylim(-500, 10)

    # Clearer view of frequency response at low cutoff frequency
    ax1 =  plt.axes([0.2, 0.3, 0.2, 0.2])
    ax1.plot((fs * 0.5 / np.pi) * w, 20 * np.log10(abs(h)), 'r')
    plt.title('Zoom in low cutoff')   
    plt.xlim(0,100)
    plt.ylim(-200, 10)

    # Clearer view of frequency response at high cutoff frequency
    ax2 =  plt.axes([0.6, 0.3, 0.2, 0.2])
    ax2.plot((fs * 0.5 / np.pi) * w, 20 * np.log10(abs(h)), 'r')
    plt.title('Zoom in high cutoff')   
    plt.xlim(5000,8000)
    plt.ylim(-10, 10)
    
    plt.grid(True)
    plt.show()

    zi = lfilter_zi(b, a) # set initial state of the filter
    z, _ = lfilter(b, a, audio_readed, zi=zi*audio_readed[0]) # filter signal from left to right
    z2, _ = lfilter(b, a, z, zi=zi*z[0]) # filter signal from right to left
    audio_filtered = filtfilt(b, a, audio_readed) # filter signal forward and backward

    filtered_audio.append(filename) # save name and filtered audio in list
    filtered_audio.append(audio_filtered) # save audio filtered in list
    filtered_audio.append(fs) # save fs in list
    

    # define graphics parameters 
    plt.rcParams['figure.figsize'] = [15, 7]
    plt.tight_layout(pad=3.0)

    # show graph with original audio signal
    plt.plot(time_paded, audio_readed, 'r', linewidth = 3)

    # show graph with filtered audio signal
    plt.plot(time_paded,audio_filtered, 'g', alpha = 0.8)

    plt.legend(('original signal','Filtered signal'), loc = 3, prop={'size': 12})  
    plt.grid(True)
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    plt.title(filename.split('/')[1] + ' saying ' +  filename.split('/')[-2], fontsize = 16, fontweight = 'bold')

    plt.show()

    

    print('All audios are filtered')
    return filtered_audio


# call function to visualize frequency response and graphic of each filtered audio
filtered_audio = filter_original_voice('Dataset/Andre/casa/0.wav') 

### View spectrogram to see where are the power and silence in filtered signal


In [ ]:
# import required packages
def see_spectrogram_scipy():
    
    name = filtered_audio[0] # get the word and the person
    wave = filtered_audio[1] # get audio wave
    fs = filtered_audio[2] # get sample rate
        
    
    plt.rcParams['figure.figsize'] = [15, 15]
    plt.tight_layout(pad=3.0)

    # perfom spectogram analysis
    f, t, S1 = spectrogram(wave, fs ,window='flattop', nperseg=fs//10, noverlap=fs//20, scaling='density', mode='magnitude')

    # show graph with spectogram of each audio
    plt.pcolormesh(t, f[:800], S1[:800][:])
    plt.xlabel('time(s)')
    plt.ylabel('frequency(Hz)')
    plt.title(name.split('/')[1]+ ' saying ' + name.split('/')[-2], fontsize = 16, fontweight = 'bold')

    plt.show()
     
        
def see_spectrogram_librosa (y_scale = 'log'):
            
    name = filtered_audio[0] # get the word and the person
    wave = filtered_audio[1] # get audio wave
    fs = filtered_audio[2] # get sample rate


    X = stft(wave) # aply stft to filtered signal
    Xdb = amplitude_to_db(abs(X)) # convert signal only to positive values and convert amplitude to db

    # define ghap parameters 
    plt.rcParams['figure.figsize'] = [15, 7]
    plt.tight_layout(pad=3.0)

    # show graph with spectogram of each audio
    display.specshow(Xdb[:800], sr=fs, x_axis='time', y_axis= y_scale)
    plt.title(name.split('/')[1]+ ' saying ' + name.split('/')[-2], fontsize = 16, fontweight = 'bold')
    plt.colorbar()
    
# see_spectrogram_librosa()
see_spectrogram_scipy()

### Split the audio in silence and non-silence

In [ ]:
def find_non_consecutive(lst, distance = 3):
    
    # for loop to iterate the list   
    for i,j in enumerate(lst,lst[0]): 
        
        # get the non consecutive values which represent words in the audio
        if i!=j and j - i > distance: 
            return [i,j], j
                

def calculate_rms(show_graph = False):  
    
    get_exp_value = False # bool variable to indicate if exponential value is already known
    audio_flag= []
    frame_length = 2048
    hop_length = 1024
    
    name = filtered_audio[0] # get the word and the person
    wave = filtered_audio[1] # get audio wave
    fs = filtered_audio[2] # get sample rate
        
    # determine average power in each frame
    rms = feature.rms(y=abs(wave), frame_length=frame_length, hop_length=hop_length, center=True)[0]

    rms_rounded = np.round(rms, decimals=2) # round rms values in 2 decimals

    if not get_exp_value:
        difference_value = (len(wave)/len(rms_rounded))
        x = math.floor(math.log10(difference_value))  # x = 3
        exp_value = 1 * 10**x                       
        get_exp_value = True

    # list to return values where audio is
    silence_time = []
    for rms_index in range(len(rms_rounded)):
            if rms_rounded[rms_index] == 0:
                silence_time.append(rms_index)

    marker_phrase, index = find_non_consecutive(silence_time) # get the limits of phrase in audio segment    
    marker_words_limit = silence_time.index(index) # find the index of the limit in marker phrase

    del silence_time[:marker_words_limit] # delete all values in list until the limit to research
    revaluate_word_position, _ = find_non_consecutive(silence_time) # get the limits of word in audio segment

    phrase_begin = np.multiply(np.min(marker_phrase), exp_value) # get the value where phrase begin
    phrase_end = np.multiply(np.max(marker_phrase), exp_value) # get the value where phrase end

    word_begin = np.multiply(np.min(revaluate_word_position), exp_value) # get the value where word begin     
    word_end = np.multiply(np.max(revaluate_word_position),exp_value) # get the value where word end

    audio_flag.append(phrase_begin)    
    audio_flag.append(phrase_end)   
    audio_flag.append(word_begin)   
    audio_flag.append(word_end)   

    # get number of points in signal and the respective time
    frames_f = range(len(rms_rounded))
    time_scale = frames_to_time(frames_f, sr=fs, hop_length=hop_length)

    # define ghap parameters 
    plt.rcParams['figure.figsize'] = [15, 7]
    plt.tight_layout(pad=3.0)

    # show graph with rms of each audio
    plt.plot(time_scale, rms_rounded, 'g')
    plt.title(name.split('/')[1]+ ' saying ' + name.split('/')[-2], fontsize = 16, fontweight = 'bold')


    plt.show()

    return audio_flag

audio_flag = calculate_rms()

## Extract number of syllables in audio

In [ ]:
def split_audio_syllables():
     
    splited_phonemes = []    
        
    name = filtered_audio[0] # get the word and the person
    wave = filtered_audio[1] # get audio wave
    fs = filtered_audio[2] # get sample rate
    
    y = wave[audio_flag[2]:audio_flag[3]] # get only the word

    notes_changes = onset.onset_strength(y=y, sr=fs, hop_length=512) # extract onset transitions
    ac = autocorrelate(notes_changes/notes_changes.max(), max_size= fs // 512) # aply autocorrelation to onset transitions
    peaks, _ = find_peaks(ac, height=0.15) # find peak with a minimum height

    if len(peaks):
        syllable_limit =  audio_flag[2] + math.floor(peaks[-1]/2) * 1000 + 1000 # divid the audio based on the peak from autocorrelation
    else:
        syllable_limit =  audio_flag[2] + math.floor((len(y)/1000)/2) * 1000 + 1000 

    splited_phonemes.append(audio_flag[2]) # save the time where first syllable beggin
    splited_phonemes.append(syllable_limit) # save the time where first syllable end or second syllable beggin
    splited_phonemes.append(audio_flag[3]) # save the time where second syllable end
    splited_phonemes.append(name) 
    
    fig, ax = plt.subplots()
    
    ax.plot(ac)
    ax.set(title='Auto-correlation', xlabel='Lag (frames)')
    plt.plot(peaks, ac[peaks], "x")
    plt.show()
        
    print('All words are splited in phonemes')
    
    return splited_phonemes

splited_phonemes = split_audio_syllables()

### Let's listen the phonemes


In [ ]:
wave = filtered_audio[1] # get audio wave

'''change the line below to listen the two phonemes like that:
splited_phonemes[0]:splited_phonemes[1] or splited_phonemes[1]:splited_phonemes[2]'''

y = wave[splited_phonemes[1]:splited_phonemes[2]] # listen only the individual phoneme
ipd.Audio(y,rate=22050)

#### Normalize audio values after pre-processement


In [ ]:
def normalize_audio(audio, name):
    positive_audio = abs(audio) # convert audio to absolute values
    audio = positive_audio / np.max(positive_audio) # divide audio values by max value in audio to normalize
    
    # define graph parameters and plot graph
    plt.figure(figsize=(15,4))
    plt.plot(np.linspace(0, len(positive_audio) / 22050, num=len(positive_audio)), audio)
    plt.title(f'Audio Normalization of {name}', fontsize = 16, fontweight = 'bold')
    plt.grid(True)

    return audio

#### Aply framimg in audio

In [ ]:
def framing_audio(audio, FFT_size=1024, hop_size=10, sample_rate=22050):
  
    audio = np.pad(audio, int(FFT_size / 2), mode='reflect') # padding the signal to ensure that all samples in frame have equal lenght 
    frame_len = np.round(sample_rate * hop_size / 1000).astype(int) # convert frame lenght from seconds to samples
    frame_num = int((len(audio) - FFT_size) / frame_len) + 1 # convert frame number from seconds to samples
    frames = np.zeros((frame_num,FFT_size)) # define structure of frames
    
    for n in range(frame_num):
        frames[n] = audio[n*frame_len:n*frame_len+FFT_size]
    
    return frames

def fft_spectrum(audio_framed, hop_size=10 , FFT_size=1024):    

    ''' create window to limit audio frammed
        hann window is used because this method is useful
        to smoothing discontinuities in the edges of each frame'''
    
    window = get_window("hann", FFT_size, fftbins=True) 
    plt.figure(figsize=(15,4))
    plt.plot(window)
    plt.grid(True)
    plt.title('Window applied to frammimg')
    plt.show()

    audio_windowed = audio_framed * window # aply the smooth operation in audio framed
    
    
    plt.figure(figsize=(15,6))
    plt.plot(audio_framed[0], label='Original Frame' )
    plt.plot(audio_windowed[0], label='Frame After Windowing' )
    plt.grid(True)
    plt.legend()
    plt.title('Example of framming one frame')
    plt.show

    audio_windowedT = np.transpose(audio_windowed) # aply the tranpose to performed the fft
    audio_fft = np.empty((int(1 + FFT_size // 2), audio_windowedT.shape[1]), dtype=np.complex64, order='F') # define structure of fft
    for n in range(audio_fft.shape[1]):
        audio_fft[:, n] = fft.fft(audio_windowedT[:, n], axis=0)[:audio_fft.shape[0]]

    audio_fft = np.transpose(audio_fft) # invert the shape of resultant fft to the same of audio_windowed

    audio_power = np.square(np.abs(audio_fft)) # get the power of signal
    
    return audio_power

#### Get the maximum and minimum frequencies of spectrum

In [ ]:
def maximum_minimum_hz(audio):
        
    S, _ = magphase(stft(y=audio, n_fft=1024, hop_length=10)) # perform stft in the signal
    spec_bw = feature.spectral_bandwidth(S=S) # obtain the spectral band to get max and min frequencies
    
    fig, ax = plt.subplots(nrows=2, sharex=True)
    times = times_like(spec_bw)
    
    ax[0].semilogy(times, spec_bw[0], label='Spectral bandwidth')
    ax[0].set(title='Band frequencies')
    ax[0].set(ylabel='Hz', xticks=[], xlim=[times.min(), times.max()])
    ax[0].legend()

    display.specshow(amplitude_to_db(S, ref=np.max),
                             y_axis='log', x_axis='time', ax=ax[1])
    plt.plot(times, spec_bw[0], label='Spectral centroid', color='w')
    plt.legend(loc='lower right')
    plt.title('Power Spectrogram')
    
    return np.round(np.min(spec_bw), decimals=4), np.round(np.max(spec_bw), decimals=4)

audio = maximum_minimum_hz(wave[splited_phonemes[1]:splited_phonemes[2]])

#### Construction of filter bank

In [ ]:
# convert hz to mel scale
def freq_to_mel(freq):
    return np.round(1125 * ln(1.0 + freq / 700), decimals=4)

# convert mel scale to hz
def mel_to_freq(mels):
    return 700 * (exp(mels / 1125) - 1)


def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, sample_rate=22050):
    freqs = []
    filter_bank = []
    
    fmin_mel = freq_to_mel(fmin) # convert initial frequency limit from Hz to mel
    fmax_mel = freq_to_mel(fmax) # convert final frequency limit from Hz to mel
    
    print(f'Minimum mel frequency: {fmin_mel}')
    print(f'Maximum mel frequency: {fmax_mel}')
    
    mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num + 2) # construct linear array between the frequencies limits
    
    [freqs.append(mel_to_freq(mel)) for mel in mels]  # convert the linear array from mel scale to Hz    
    [filter_bank.append(np.floor((FFT_size + 1) / sample_rate * freq).astype(int)) for freq in freqs]  # normalize values to FFT size

    filters = np.zeros((len(filter_bank)-2,int(FFT_size/2+1))) # define the structure of filters

    for n in range(len(filter_bank)-2):
        
        #define the lower limit of the filter
        filters[n, filter_bank[n] : filter_bank[n + 1]] = np.linspace(0, 1, filter_bank[n + 1] - filter_bank[n]) # this function will produce a come up from 0 to 1 in the begining point until the after
        
        #define the upper limit of the filter
        filters[n, filter_bank[n + 1] : filter_bank[n + 2]] = np.linspace(1, 0, filter_bank[n + 2] - filter_bank[n + 1]) # this function will produce a come down from 1 to 0 in the final point until the afte
        
    plt.figure(figsize=(15,4))
    for n in range(filters.shape[0]):
        plt.plot(filters[n])
        plt.xlim(filter_bank[0]-5,filter_bank[-1]+5)    
        plt.title('Filters bank')
    
    # saw in the librosa library
    freqs = np.array(freqs) # convert list to array
    enorm = 2.0 / (freqs[2:mel_filter_num+2] - freqs[:mel_filter_num]) # divide the triangular MEL filter by the width of the MEL band (area normalization)
    filters *= enorm[:, np.newaxis]
    
    plt.figure(figsize=(15,4))
    for n in range(filters.shape[0]):
        plt.plot(filters[n])
        plt.xlim(filter_bank[0]-5,filter_bank[-1]+5)
        plt.title('Filters bank normalized')
        
        
    return filters

#### Calculation of mfccs

In [ ]:
def mfccs(filters, audio_power, phoneme):
        
    audio_filtered_mfccs = np.dot(filters, np.transpose(audio_power)) # aply the filters into the spectrum calculated above
    audio_log = 10.0 * np.log10(audio_filtered_mfccs) # convert to log scale
    audio_log.shape


    mel_coeficients = 20

    coefficients = np.transpose(fft.dct(audio_log, type=3)) # will be calculated the type 3 dct to distinguish from high to low frequencies
    mfccs_values = np.dot(coefficients, audio_log) # aply the number of coeficients in the spectrum

    plt.figure(figsize=(15,5))
    plt.imshow(mfccs_values, aspect='auto', origin='lower',cmap='coolwarm')
    plt.savefig(phoneme)
    plt.title(f'Mfcc graph of phoneme {phoneme}')
    
    predict = evaluate_model(phoneme)
    
    return predict

## Let's identify the phonemes

In [ ]:
def split_written_word(word):
    # this function is to split word in syllables
    dic = pyphen.Pyphen(lang='pt')
    return dic.inserted(word).split('-')

def split_phoneme_to_test():
    hop_size = 10 # ms
    FFT_size = 1024 # size of frame
    pos = 0
    phonemes = []
    
    wave = filtered_audio[1] # get audio wave
    name = filtered_audio[0] # get the word and the person
    name = name.split('/')[-2] # get the word said
    phonetics = split_written_word(name) # split the word in syllables

    while pos != len(splited_phonemes) - 2:
        if pos == 0:
            phoneme = phonetics[0]
        else:
            phoneme = phonetics[1]
                
        audio = wave[splited_phonemes[pos]:splited_phonemes[pos+1]] # get the wave per syllable
        
        audio_normalized = normalize_audio(audio, phoneme) # normalize audio
        
        audio_framed = framing_audio(audio_normalized, FFT_size=FFT_size, hop_size=hop_size, sample_rate=22050) # framming the audio
        
        audio_power = fft_spectrum(audio_framed, hop_size=hop_size, FFT_size=FFT_size) # perform fft and convert to power

        freq_min, freq_max = maximum_minimum_hz(audio) # get the minimun and the maximum frequencies of the audio
        
        filters_to_mfcc =  get_filter_points(freq_min, freq_max, mel_filter_num=10, FFT_size=FFT_size, sample_rate=22050) # construct filter bank

        predict = mfccs(filters_to_mfcc, audio_power, phoneme) # calculate the mfccs  

        phonemes.append(predict)
                
        pos += 1
        
    return phonemes 

# <center> <span style='color :blue' >Get the result from phoneme and the speaker recognition</span>

# Let's test the trained model

In [ ]:
def evaluate_model(phoneme):
    
    model = pickle.load(open('final_model.sav', 'rb')) # load model

    example = cv2.imread(phoneme +'.png', cv2.IMREAD_GRAYSCALE)
    X_test = cv2.resize(example,[360,480])
    
    X_test = np.array(X_test).reshape(1,-1) # reshape data to the model
    X_test = X_test/255.0 # normalize data
    
    prediction = model.predict(X_test) # test model

    return prediction

### Calculate the fundamental frequency of each person

In [ ]:
def p_yin(y):
    
    frame_length = 2048
    f0, voiced_flag, voiced_probs = pyin(y=y, fmin=80,fmax=500,frame_length=frame_length) # extract value from pyin
    
    times = times_like(f0)
    D = amplitude_to_db(np.abs(stft(y)), ref=np.max)
    
    fig, ax = plt.subplots()
    img = display.specshow(D, x_axis='time', y_axis='log', ax=ax)
    ax.set(title='pYIN fundamental frequency estimation')
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    ax.plot(times, f0, label='f0', color='cyan', linewidth=4)
    ax.legend(loc='upper right')

    f0 = np.mean(f0[np.where(voiced_flag == True)]) # calculate the mean of f0 values
    
    y_harmonic, y_percussive = effects.hpss(y) # extract percurssive and harmonics from the audio
    y_harmonic = np.round(np.mean(y_harmonic), decimals=5) # calculate the mean of harmonics values
    y_percussive = np.round(np.mean(y_percussive), decimals=6) # calculate the mean of percurssives values
           
    return f0, y_harmonic, y_percussive

### Get the result of speaker identification and the word that has been said

In [ ]:
def get_result():
    predict= split_phoneme_to_test() # get the predictions of phonemes
    
    wave = filtered_audio[1] # get audio wave
    f0, harmonic, percussive = p_yin(wave[audio_flag[0]:audio_flag[1]]) # analyze the audio where exists more text
    print(f0, harmonic, percussive)
    
    phonemes = ['ca','cha', 'chu', 'da','far','la','pa', 'ri', 'sa', 'ta',
               'va', 've']
    
    help_list = []
    
    x = 0
    for i in range(len(phonemes)):
        if x <2:
            if predict[x] == phonemes[i]:
                print(i)
                help_list.append(i) 
                x += 1
        else:
            break
     
    if 140 <= f0 < 165 and harmonic == -0.0:
        person = 'André'
    elif 165 <= f0 < 193 and harmonic != -0.0:
        person = 'Marcelo'
    else:
        person = 'Unknown'
    
    sentence = person+ ' said ' +phonemes[help_list[0]]+phonemes[help_list[1]]
    print(sentence)#
    
    engine = pyttsx3.init() 
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate + 50)
    engine.say(sentence) 
    engine.runAndWait()         
    
get_result()   

# <center> <span style='color :blue' > And that is the end of the code !!!</span>
## <center> <span style='color :blue' > Thank you !!!</span>